In [33]:
import nltk
import pandas as pd
import numpy as np
import re
import  string
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joanneodannell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/joanneodannell/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/joanneodannell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
fx = pd.read_csv("Desktop/st_2195/st2195_assignment_6_data/fx.csv", parse_dates=['DATE'])
fx

,DATE,TIME PERIOD,US dollar/Euro (EXR.D.USD.EUR.SP00.A)
0,1999-01-04,04 Jan 1999,1.1789
1,1999-01-05,05 Jan 1999,1.1790
2,1999-01-06,06 Jan 1999,1.1743
3,1999-01-07,07 Jan 1999,1.1632
4,1999-01-08,08 Jan 1999,1.1659
...,...,...,...
6829,2025-06-09,09 Jun 2025,1.1410
6830,2025-06-10,10 Jun 2025,1.1429
6831,2025-06-11,11 Jun 2025,1.1433
6832,2025-06-12,12 Jun 2025,1.1594


In [9]:
speeches = pd.read_csv("Desktop/st_2195/st2195_assignment_6_data/speeches.csv", delimiter='|', parse_dates=['date'])
speeches

,date,speakers,title,subtitle,contents
0,2025-05-26,Christine Lagarde,Earning influence: lessons from the history of...,"Speech by Christine Lagarde, President of the ...",SPEECH Earning influence: lessons from the ...
1,2025-05-23,Philip R. Lane,Inflation and disinflation in the euro area,"Lecture by Philip R. Lane, Member of the Execu...",NaN
2,2025-05-22,Frank Elderson,"Nature’s bell tolls for thee, economy!","Keynote speech by Frank Elderson, Member of th...","SPEECH Nature’s bell tolls for thee, econom..."
3,2025-05-21,NaN,"Discussion of ""Negative interest rates and the...","Presentation by Philip R. Lane, Member of the ...",NaN
4,2025-05-16,Philip R. Lane,The communication of monetary policy decisions...,"Remarks by Philip R. Lane, Member of the Execu...",REMARKS The communication of monetary polic...
...,...,...,...,...,...
2899,1997-05-13,Alexandre Lamfalussy,The European Central Bank: independent and acc...,Keynote speech delivered by Alexandre Lamfalus...,The European Central Bank: independent and a...
2900,1997-04-30,Alexandre Lamfalussy,The operation of monetary policy in stage thre...,"Address by Alexandre Lamfalussy, President of ...",The operation of monetary policy in stage th...
2901,1997-04-22,Alexandre Lamfalussy,Convergence and the role of the European Centr...,"Remarks by Alexandre Lamfalussy, President of ...",Convergence and the role of the European Cen...
2902,1997-03-10,Alexandre Lamfalussy,Securing the benefits of EMU,"Address by Alexandre Lamfalussy, President of ...",Securing the benefits of EMU Address by Al...


In [11]:
fx.columns = fx.columns.str.strip().str.lower()
fx.rename(columns={'us dollar/euro (exr.d.usd.eur.sp00.a)': 'usd_eur'}, inplace=True)
fx

,date,time period,usd_eur
0,1999-01-04,04 Jan 1999,1.1789
1,1999-01-05,05 Jan 1999,1.1790
2,1999-01-06,06 Jan 1999,1.1743
3,1999-01-07,07 Jan 1999,1.1632
4,1999-01-08,08 Jan 1999,1.1659
...,...,...,...
6829,2025-06-09,09 Jun 2025,1.1410
6830,2025-06-10,10 Jun 2025,1.1429
6831,2025-06-11,11 Jun 2025,1.1433
6832,2025-06-12,12 Jun 2025,1.1594


In [13]:
# Merge on date (only keep dates in fx.csv)
df = pd.merge(fx,speeches, on="date", how="left")
df[df['contents'].notna()]

,date,time period,usd_eur,speakers,title,subtitle,contents
8,1999-01-14,14 Jan 1999,1.1653,Willem F. Duisenberg,The euro has arrived,Speech by the President of the European Centra...,The euro has arrived Speech by the Preside...
9,1999-01-15,15 Jan 1999,1.1626,Willem F. Duisenberg,European economic and monetary union - latest ...,Speech by Willem F. Duisenberg President of th...,European economic and monetary union - lates...
10,1999-01-18,18 Jan 1999,1.1612,Willem F. Duisenberg,Hearing at the European Parliament's Sub-Commi...,Introductory statement delivered by Dr. Willem...,Hearing at the European Parliament's Sub-Com...
15,1999-01-25,25 Jan 1999,1.1584,Sirkka Hämäläinen,Finnish savers and investors in the euro area,"Speech presented by Ms Sirkka Hämäläinen, Memb...",Finnish savers and investors in the euro are...
16,1999-01-25,25 Jan 1999,1.1584,Willem F. Duisenberg,Monetary policy in the euro area,Speech by the President of the European Centra...,Monetary policy in the euro area Speech by...
...,...,...,...,...,...,...,...
7483,2025-05-15,15 May 2025,1.1185,Luis de Guindos,Financial stability in the euro area,"Speech by Luis de Guindos, Vice-President of t...",SPEECH Financial stability in the euro area...
7484,2025-05-15,15 May 2025,1.1185,Piero Cipollone,Harnessing the digital future of payments: Eur...,"Speech by Piero Cipollone, Member of the Execu...",SPEECH Harnessing the digital future of pay...
7485,2025-05-16,16 May 2025,1.1194,Philip R. Lane,The communication of monetary policy decisions...,"Remarks by Philip R. Lane, Member of the Execu...",REMARKS The communication of monetary polic...
7489,2025-05-22,22 May 2025,1.1309,Frank Elderson,"Nature’s bell tolls for thee, economy!","Keynote speech by Frank Elderson, Member of th...","SPEECH Nature’s bell tolls for thee, econom..."


In [15]:
# Remove outliers in exchange rate
print(len(df))
df = df[(df['usd_eur'] > 0) & (df['usd_eur'] < 10)]
print(len(df))

7506
7444


In [17]:
# Forward fill missing exchange rate and drop rows with remaining NAs
df['usd_eur'] = df['usd_eur'].ffill()
df = df.dropna(subset=['usd_eur'])
df[df['contents'].notna()]

,date,time period,usd_eur,speakers,title,subtitle,contents
8,1999-01-14,14 Jan 1999,1.1653,Willem F. Duisenberg,The euro has arrived,Speech by the President of the European Centra...,The euro has arrived Speech by the Preside...
9,1999-01-15,15 Jan 1999,1.1626,Willem F. Duisenberg,European economic and monetary union - latest ...,Speech by Willem F. Duisenberg President of th...,European economic and monetary union - lates...
10,1999-01-18,18 Jan 1999,1.1612,Willem F. Duisenberg,Hearing at the European Parliament's Sub-Commi...,Introductory statement delivered by Dr. Willem...,Hearing at the European Parliament's Sub-Com...
15,1999-01-25,25 Jan 1999,1.1584,Sirkka Hämäläinen,Finnish savers and investors in the euro area,"Speech presented by Ms Sirkka Hämäläinen, Memb...",Finnish savers and investors in the euro are...
16,1999-01-25,25 Jan 1999,1.1584,Willem F. Duisenberg,Monetary policy in the euro area,Speech by the President of the European Centra...,Monetary policy in the euro area Speech by...
...,...,...,...,...,...,...,...
7483,2025-05-15,15 May 2025,1.1185,Luis de Guindos,Financial stability in the euro area,"Speech by Luis de Guindos, Vice-President of t...",SPEECH Financial stability in the euro area...
7484,2025-05-15,15 May 2025,1.1185,Piero Cipollone,Harnessing the digital future of payments: Eur...,"Speech by Piero Cipollone, Member of the Execu...",SPEECH Harnessing the digital future of pay...
7485,2025-05-16,16 May 2025,1.1194,Philip R. Lane,The communication of monetary policy decisions...,"Remarks by Philip R. Lane, Member of the Execu...",REMARKS The communication of monetary polic...
7489,2025-05-22,22 May 2025,1.1309,Frank Elderson,"Nature’s bell tolls for thee, economy!","Keynote speech by Frank Elderson, Member of th...","SPEECH Nature’s bell tolls for thee, econom..."


In [19]:
# Calculate return
df['usd_eur_return'] = df['usd_eur'].pct_change() * 100
df['good_news'] = (df['usd_eur_return'] > 0.5).astype(int)
df['bad_news'] = (df['usd_eur_return'] < -0.5).astype(int)
df

,date,time period,usd_eur,speakers,title,subtitle,contents,usd_eur_return,good_news,bad_news
0,1999-01-04,04 Jan 1999,1.1789,NaN,NaN,NaN,NaN,NaN,0,0
1,1999-01-05,05 Jan 1999,1.1790,NaN,NaN,NaN,NaN,0.008482,0,0
2,1999-01-06,06 Jan 1999,1.1743,NaN,NaN,NaN,NaN,-0.398643,0,0
3,1999-01-07,07 Jan 1999,1.1632,NaN,NaN,NaN,NaN,-0.945244,0,1
4,1999-01-08,08 Jan 1999,1.1659,NaN,NaN,NaN,NaN,0.232118,0,0
...,...,...,...,...,...,...,...,...,...,...
7501,2025-06-09,09 Jun 2025,1.1410,NaN,NaN,NaN,NaN,-0.008763,0,0
7502,2025-06-10,10 Jun 2025,1.1429,NaN,NaN,NaN,NaN,0.166521,0,0
7503,2025-06-11,11 Jun 2025,1.1433,NaN,NaN,NaN,NaN,0.034999,0,0
7504,2025-06-12,12 Jun 2025,1.1594,NaN,NaN,NaN,NaN,1.408204,1,0


In [21]:
# Drop empty contents
df = df.dropna(subset=['contents'])
df

,date,time period,usd_eur,speakers,title,subtitle,contents,usd_eur_return,good_news,bad_news
8,1999-01-14,14 Jan 1999,1.1653,Willem F. Duisenberg,The euro has arrived,Speech by the President of the European Centra...,The euro has arrived Speech by the Preside...,-0.774864,0,1
9,1999-01-15,15 Jan 1999,1.1626,Willem F. Duisenberg,European economic and monetary union - latest ...,Speech by Willem F. Duisenberg President of th...,European economic and monetary union - lates...,-0.231700,0,0
10,1999-01-18,18 Jan 1999,1.1612,Willem F. Duisenberg,Hearing at the European Parliament's Sub-Commi...,Introductory statement delivered by Dr. Willem...,Hearing at the European Parliament's Sub-Com...,-0.120420,0,0
15,1999-01-25,25 Jan 1999,1.1584,Sirkka Hämäläinen,Finnish savers and investors in the euro area,"Speech presented by Ms Sirkka Hämäläinen, Memb...",Finnish savers and investors in the euro are...,0.146970,0,0
16,1999-01-25,25 Jan 1999,1.1584,Willem F. Duisenberg,Monetary policy in the euro area,Speech by the President of the European Centra...,Monetary policy in the euro area Speech by...,0.000000,0,0
...,...,...,...,...,...,...,...,...,...,...
7483,2025-05-15,15 May 2025,1.1185,Luis de Guindos,Financial stability in the euro area,"Speech by Luis de Guindos, Vice-President of t...",SPEECH Financial stability in the euro area...,-0.258605,0,0
7484,2025-05-15,15 May 2025,1.1185,Piero Cipollone,Harnessing the digital future of payments: Eur...,"Speech by Piero Cipollone, Member of the Execu...",SPEECH Harnessing the digital future of pay...,0.000000,0,0
7485,2025-05-16,16 May 2025,1.1194,Philip R. Lane,The communication of monetary policy decisions...,"Remarks by Philip R. Lane, Member of the Execu...",REMARKS The communication of monetary polic...,0.080465,0,0
7489,2025-05-22,22 May 2025,1.1309,Frank Elderson,"Nature’s bell tolls for thee, economy!","Keynote speech by Frank Elderson, Member of th...","SPEECH Nature’s bell tolls for thee, econom...",-0.105998,0,0


In [23]:
# Tokenization helper
def clean_text(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", " ", text)
    tokens = word_tokenize(text)
    words = [w for w in tokens if w.isalpha() and w not in stopwords.words('english')]
    return words
    # Get top 20 words
def get_top_words(df_sub):
    all_words = df_sub['contents'].apply(clean_text).sum()
    freq = Counter(all_words).most_common(20)
    return pd.DataFrame(freq, columns=["word", "count"])

In [35]:
good_news = get_top_words(df[df["good_news"] == 1])
good_news

,word,count
0,euro,5535
1,policy,5115
2,financial,4874
3,monetary,4579
4,de,4381
5,area,3797
6,ecb,3070
7,economic,2891
8,inflation,2847
9,central,2835


In [37]:
bad_news = get_top_words(df[df["bad_news"] == 1])
bad_news

,word,count
0,euro,6515
1,financial,5353
2,policy,5270
3,monetary,4827
4,area,4528
5,de,3397
6,banks,3274
7,central,3106
8,market,3086
9,ecb,3054


In [41]:
# Export
good_news.to_csv("Desktop/st_2195/st2195_assignment_6/good_indicators.csv", index=False)
bad_news.to_csv("Desktop/st_2195/st2195_assignment_6/bad_indicators.csv", index=False)